#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo
import json

In [52]:
client = pymongo.MongoClient("mongodb://192.168.1.10:27017/")
database = client['exercices']
collection = database['kickstarter']

In [53]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [55]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [56]:
df_ks = df_ks.drop(columns=['pledged', 'backers', 'usd pledged'])

In [57]:
df_ks = df_ks.drop(66141)

In [58]:
df_ks['launch_year'] = (df_ks['launched'].str[0:4]).astype('int')

## Question 0

### Netoyer les données

In [60]:
df_ks.isna().sum()

ID                  0
name                2
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
state               0
country             0
usd_pledged_real    0
launch_year         0
dtype: int64

In [61]:
df_ks = df_ks.dropna()

In [62]:
df_ks = df_ks.rename(columns={"ID": "_id"})

### Importer les données

In [63]:
collection.insert_many(df_ks.to_dict('records'))

In [64]:
cursor = collection.find()

In [65]:
next(cursor)

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': '2011-08-17 06:31:31',
 'state': 'canceled',
 'country': 'US',
 'usd_pledged_real': 1145.0,
 'launch_year': 2011}

## Question 1  

In [66]:
cur = collection.find().sort([("usd_pledged_real", -1)])
for i in range(5):
    print(next(cur))
    print()

{'_id': 342886736, 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2014-08-30', 'goal': '50000.0', 'launched': '2014-07-08 10:14:37', 'state': 'successful', 'country': 'US', 'usd_pledged_real': 13285226.36, 'launch_year': 2014}

{'_id': 2103598555, 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2016-06-30', 'goal': '1000000.0', 'launched': '2016-05-24 15:49:52', 'state': 'successful', 'country': 'US', 'usd_pledged_real': 12779843.49, 'launch_year': 2016}

{'_id': 1033978702, 'name': 'OUYA: A New Kind of Video Game Console', 'category': 'Gaming Hardware', 'main_category': 'Games', 'currency': 'USD', 'deadline': '2012-08-09', 'goal': '950000.0', 'launched': '2012-07-10 14:44:41', 'state': 'successful', 'country': 'US', 'usd_pledged_real': 8596474.58, 'launch_year': 2012}

{'_id': 45009942

## Question 2

In [67]:
cur = collection.aggregate([{"$group" : {"_id" : "$state", "count" : {"$sum" : 1}}}])
for document in cur:
    if document['_id'] == 'successful' : print('il y a' ,document['count'] , 'projets qui ont réussi')
# 53040

il y a 53040 projets qui ont réussi


## Question 3

In [68]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "count" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Accessories', 'count': 1204},
 {'_id': 'Apps', 'count': 2535},
 {'_id': 'Ready-to-wear', 'count': 332},
 {'_id': 'Music', 'count': 6229},
 {'_id': 'Literary Journals', 'count': 118},
 {'_id': 'Drama', 'count': 871},
 {'_id': 'Workshops', 'count': 59},
 {'_id': 'Video Art', 'count': 65},
 {'_id': 'Classical Music', 'count': 1064},
 {'_id': 'Art Books', 'count': 1065},
 {'_id': 'Events', 'count': 322},
 {'_id': 'Pop', 'count': 1302},
 {'_id': 'Horror', 'count': 525},
 {'_id': 'Faith', 'count': 439},
 {'_id': 'Printing', 'count': 83},
 {'_id': 'Jewelry', 'count': 472},
 {'_id': 'Installations', 'count': 178},
 {'_id': 'Design', 'count': 1641},
 {'_id': 'Product Design', 'count': 8886},
 {'_id': 'Crafts', 'count': 1834},
 {'_id': 'Candles', 'count': 168},
 {'_id': 'Art', 'count': 3358},
 {'_id': 'Web', 'count': 2017},
 {'_id': 'Bacon', 'count': 78},
 {'_id': 'Romance', 'count': 74},
 {'_id': 'Fashion', 'count': 3379},
 {'_id': 'Hardware', 'count': 1431},
 {'_id': 'R&B', 'count': 

## Question 4

In [69]:
cur = collection.aggregate([{"$match":{"launch_year":{"$lt":2016}}},{"$group":{"_id":"$country","count":{"$sum":1}}}])
for document in cur:
    if document['_id'] == 'FR' : print('Il y a', document['count'], 'projets lancé avant 2016 en France')
# 330

Il y a 330 projets lancé avant 2016 en France


## Question 5

In [71]:
cur = collection.aggregate([{"$match":{"goal":{"$gt":200000}}},{"$group":{"_id":"$country","count":{"$sum":1}}}])
for document in cur:
    if document['_id'] == 'US' : print('Il y a', document['count'], 'projets américains ayant demandé plus de 200 000 dollars')
# 1026

Il y a 1026 projets américains ayant demandé plus de 200 000 dollars


## Question 6 

In [72]:
cur = collection.find({"name" : {"$regex" : "Sport"}})
count = 0
for document in cur:
    count += 1
print('Il y a', count, 'projets avec le mot "Sport" dedans')
# 323

Il y a 323 projets avec le mot "Sport" dedans
